# Energy Analysis

This notebook analysis the `.rld` files from the RocketLogger energy measurement. It analysis the digital traces to mark the beginning and end of each inference and layer.
The resulting Dataframe only contains the averaged result of a single layer.


For file size reasons the original `.rld` files are not included in this repo.

The results are already in `results_energy`.

For the usage of the RocketLoggerData API see [here](https://rocketlogger.ethz.ch/python/v1.1.6/).

- **DI1:** Inference Indicator
- **DI2:** Layer toggle
- **DI3:** Waiting for input

In [6]:
from rocketlogger.data import RocketLoggerData
import numpy as np
import pandas as pd
from datetime import datetime, date
import json
import glob
import collections

In [4]:
%matplotlib widget
rld.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<module 'matplotlib.pyplot' from '/usr/local/lib/python3.8/site-packages/matplotlib/pyplot.py'>

In [7]:
# cut data
sample_step = 1/64000

start_in_s = 0.01
stop_in_s = -1

cut_start = int(start_in_s // sample_step)
cut_start = 0
cut_stop = -1

In [ ]:
filename = 'energy_measurements/20200805_102739_inference-energy_measurement.rld'

rld = RocketLoggerData(filename)
comment = rld.get_comment()
print(comment)
energy_inference_table, energy_inference_layer_table = energy_analysis(rld, filename)

In [8]:
filenames = glob.glob('energy_measurements/*.rld')
df = pd.DataFrame()
df_layers = pd.DataFrame()

df = pd.concat([df, ])
for file in filenames:
    print("\n----\nNext ...")
    rld = RocketLoggerData(file)
    comment = rld.get_comment()
    ### Manually filter
#     if ('F4' not in comment) or ('ResNet' not in comment) or ('float32' not in comment):
#         continue
#         print("Skipping ...")
    print(comment)
    
    energy_inference_table, energy_inference_layer_table = energy_analysis(rld, file)
    df = pd.concat([df, energy_inference_table])
    df_layers = pd.concat([df_layers, energy_inference_layer_table])

df.to_excel('results_energy/A_Aggregated_energy_inference_table.xlsx')
df.to_pickle('results_energy/A_Aggregated_energy_inference_table.pkl')

df_layers.to_excel('results_energy/A_Aggregated_energy_inference_layer_table.xlsx')
df_layers.to_pickle('results_energy/A_Aggregated_energy_inference_layer_table.pkl')
print("\nfin.")


----
Next ...
Read 1 file(s)
{
  "time": "2020-08-07 13:11:00.199412",
  "MCU": "NUCLEO_F767ZI",
  "model": "01d_ResNet20_CIFAR-10_none_tflite-builtins_none_none",
  "model_name": "01d_ResNet20_CIFAR-10",
  "mbed-dir": "./TFLu_benchmark-model_mbed",
  "cmsis-nn": "none",
  "pruned": 0,
  "weights": "float32",
  "activations": "float32",
  "model_type": "W-float32_A-float32",
  "compiler_optimization": "-Ofast",
  "FPU_status": 1
}
filename: energy_measurements/20200807_111342_inference-energy_measurement.rld
Detected exactly 100 or 200 inferences - continuing ...
Distribution of the layers: Counter({33.0: 166, 31.0: 34})
We detected a maximum of 33 layers per inference. Does this sound about right?
Analyzed a total of 166 inferences.

Doing some sanity checks ...
Sum of the average layer latencies:	1.6072147966867418
Average inference latency:		1.6077602597891574
Sum of the average layer energies:	0.6334499147482812
Average inference energy:		0.6336676975259649

----
Next ...
Read 1 f

In [5]:
def energy_analysis(rld, filename):
    print(f"filename: {filename}")
    rld.remove_channel('DI4')
    rld.remove_channel('DI5')
    rld.remove_channel('DI6')
    
    channels = rld.get_channel_names()
    data = {}
    data['time'] = rld.get_time()
    for channel in channels:
        data[channel] = rld.get_data(channel)
    
    
    
    no_layers = 0
    columns = ['time','filename', 'MCU', 'model', 'model_name', 'mbed-dir',
               'cmsis-nn', 'compiler_optimization', 'FPU_status',
               'model_type',
               'weights', 'activations', 'pruned',
               'no_correct_inferences',
               'latency_mean', 'latency_std',
               'voltage_mean', 'voltage_std',
               'current_mean', 'current_std',
               'power_mean', 'power_std',
               'energy_mean', 'energy_std'
              ]
    energy_inference_table = pd.DataFrame(columns=columns)

    columns = ['time', 'filename', 'MCU', 'model', 'model_name', 'mbed-dir',
               'cmsis-nn', 'compiler_optimization', 'FPU_status',
               'model_type',
               'weights', 'activations', 'pruned',
               'no_correct_inferences',
               'layer',
               'layer_latency_mean', 'layer_latency_std',
               'layer_sample_length_mean', 'layer_sample_length_std',
               'layer_voltage_mean', 'layer_voltage_std',
               'layer_current_mean', 'layer_current_std',
               'layer_power_mean', 'layer_power_std',
               'layer_energy_mean', 'layer_energy_std'
              ]
    energy_inference_layer_table = pd.DataFrame(columns=columns)

    # so here we are interested in raising and falling edges of DI1 
    # each edge marks a beginning or either an end of a single inference
    # [beg, end, beg, end, beg, end, ...]
    inference_edges = np.where(data['DI1'][:-1] != data['DI1'][1:])[0]
    no_inferences = inference_edges.shape[0] // 2
    if no_inferences == 100 or no_inferences == 200:
        print(f'Detected exactly 100 or 200 inferences - continuing ...')
    else:
        print(f'We have {no_inferences}, not as expected 100 or 200 - Aborting.')

    # check how many layers
    no_layers_arr = np.array([])
    
    for i, (i_beg, i_end) in enumerate(zip(inference_edges[0::2]+1, inference_edges[1::2]+1)):
        layer_trace = data['DI2'][i_beg-10:i_end+10]
        layer_edges = np.where(layer_trace[:-1] != layer_trace[1:])[0]
        
        # the total number of layers is the amount of toggles we detect - 1
        # basically everything toggle indicates a layer except the last toggle
        no_layers_arr= np.append(no_layers_arr, layer_edges.shape[0] - 1)
        

    #print(f"min:{no_layers_arr.min()}")
    #print(f"max:{no_layers_arr.max()}")
    print(f"Distribution of the layers: {collections.Counter(no_layers_arr)}")
    no_layers = int(no_layers_arr.max())
    
    # remove all the inferences which have not the right amount of layers
    faulty_layers_index = np.where(no_layers_arr != no_layers)[0]
    #print(faulty_layers_index)
    #print(inference_edges)
    faulty_layers_index = np.concatenate([faulty_layers_index*2, (faulty_layers_index*2)+1])
    #print(faulty_layers_index)
    inference_edges = np.delete(inference_edges, faulty_layers_index)
    #print(inference_edges)

    # update the total number of inferences
    no_inferences = inference_edges.shape[0] // 2


    print(f'We detected a maximum of {no_layers} layers per inference. Does this sound about right?')

    # writing all the model information
    # parse comment of the rocketlogger file and save the information
    model_information = json.loads(rld.get_comment())
    
    

    
    model_information['filename'] = filename
    model_information['no_correct_inferences'] = no_inferences
    # some of the earlier measurements don't have the key 'model_name'
    if not 'model_name' in model_information.keys():
        if 'LeNet' in model_information['model']:
            model_information['model_name'] = 'LeNet-MNIST'
        elif 'ResNet' in model_information['model']:
            model_information['model_name'] = '01d_ResNet20_CIFAR-10'

    inference_dict = model_information
    for _ in range(no_layers):
        energy_inference_layer_table = energy_inference_layer_table.append(model_information, ignore_index = True)

    inference_latencies= np.array([])
    inference_voltages_mean = np.array([])
    inference_voltages_std = np.array([])
    inference_currents_mean = np.array([])
    inference_currents_std = np.array([])
    inference_powers_mean = np.array([])
    inference_powers_std = np.array([])
    inference_energies = np.array([])




    layer_latencies = np.array(no_layers *[no_inferences * [np.nan]])
    layer_sample_lengths = np.array(no_layers *[no_inferences * [np.nan]])
    layer_voltages_mean = np.array(no_layers *[no_inferences * [np.nan]])
    layer_voltages_std = np.array(no_layers *[no_inferences * [np.nan]])
    layer_currents_mean = np.array(no_layers *[no_inferences * [np.nan]])
    layer_currents_std = np.array(no_layers *[no_inferences * [np.nan]])
    layer_powers_mean = np.array(no_layers *[no_inferences * [np.nan]])
    layer_powers_std = np.array(no_layers *[no_inferences * [np.nan]])

    layer_energies = np.array(no_layers *[no_inferences * [np.nan]])

    # iterate over the inferences
    # all even indices make a start, all uneven make an end
    for i, (i_beg, i_end) in enumerate(zip(inference_edges[0::2]+1, inference_edges[1::2]+1)):


        
        
        # increase the inference beginning (i_beg) by a single indice
        # the trace now starts where the gpio is up
        # and ends at the last indice where the gpio is up
        time_trace = data['time'][i_beg:i_end]
        # as we do a high side measurement our voltage is connected to the ground of the MCU
        # we measure a negative voltage -> change sign
        voltage_trace = -1 * data['V1'][i_beg:i_end]
        current_trace = data['I1H'][i_beg:i_end]

        inference_power_trace = np.multiply(voltage_trace.flatten(), current_trace.flatten())
        inference_energy = np.trapz(inference_power_trace, time_trace)

        # i_end already has the index were the gpio is low again
        inference_latencies = np.append(inference_latencies, time_trace[-1] - time_trace[0])
        inference_voltages_mean  = np.append(inference_voltages_mean , voltage_trace.mean())
        inference_voltages_std = np.append(inference_voltages_std , voltage_trace.std())
        inference_currents_mean = np.append(inference_currents_mean , current_trace.mean())
        inference_currents_std = np.append(inference_currents_std , current_trace.std())
        inference_powers_mean = np.append(inference_powers_mean , inference_power_trace.mean())
        inference_powers_std = np.append(inference_powers_std , inference_power_trace.std())
        inference_energies = np.append(inference_energies , inference_energy)


        # increase the layer trace array in case the layer gpio was toggled at the same time as the inference
        layer_trace = data['DI2'][i_beg-10:i_end+10]
        time_trace = data['time'][i_beg-10:i_end+10]
        voltage_trace = -1 * data['V1'][i_beg-10:i_end+10]
        current_trace = data['I1H'][i_beg-10:i_end+10]


        layer_edges = np.where(layer_trace[:-1] != layer_trace[1:])[0]

        #print(f"Inference #{i} is from {i_beg} to {i_end}, so a total of {i_end - i_beg}")
        #print("We do have #layers:",layer_edges.size - 1)
        #print("Layer edges", layer_edges)


        # so here we are interested in raising and falling edges
        # each edge marks a beginning and simultaneously an end
        # [beg, end/beg, end/beg, end/beg, beg/end, ..., end]
        for j, (l_beg, l_end) in enumerate(zip(layer_edges[0:-1]+1, layer_edges[1:]+1)):
            # j indicates the layer no.
            # to know the type of layer we have to check the network
            
           # print(f"\tLayer #{j} from {l_beg} to {l_end}, so a total of {l_end - l_beg}")
            

            layer_time_trace = time_trace[l_beg:l_end]
            layer_power_trace = np.multiply(voltage_trace[l_beg:l_end].flatten(), current_trace[l_beg:l_end].flatten())
            layer_energy = np.trapz(layer_power_trace, layer_time_trace)

            # use the previous resut (first time = 0) and calculate the mean

            # the calculation of the standard deviation and the updating might be wrong!

            #print(f"layer #{i}: current array {layer_latencies[i]}, new calc {time_trace[l_end] - time_trace[l_beg]}")
            try:
                layer_latencies[j][i] = layer_time_trace[-1] - layer_time_trace[0]
            except IndexError:
                print("Detected a layer which is only 1 sample long. Length will be set to the length of a single sample.")
                layer_latencies[j][i] = 1/64000
            layer_sample_lengths[j][i] = l_end - l_beg
            layer_voltages_mean[j] = voltage_trace[l_beg:l_end].mean()
            layer_voltages_std[j][i] = voltage_trace[l_beg:l_end].std()
            layer_currents_mean[j][i] = current_trace[l_beg:l_end].mean()
            layer_currents_std[j][i] = current_trace[l_beg:l_end].std()
            layer_powers_mean[j][i] = layer_power_trace.mean()
            layer_powers_std[j][i] = layer_power_trace.std()
            layer_energies[j][i] = layer_energy

    print(f"Analyzed a total of {i+1} inferences.")
    # save layer data in the table
    if no_layers > 0:
        energy_inference_layer_table['layer'] = range(no_layers)
        energy_inference_layer_table['layer_latency_mean'] = layer_latencies.mean(axis=1)
        energy_inference_layer_table['layer_latency_std'] = layer_latencies.std(axis=1)
        energy_inference_layer_table['layer_sample_length_mean'] = layer_sample_lengths.mean(axis=1)
        energy_inference_layer_table['layer_sample_length_std'] = layer_sample_lengths.std(axis=1)
        energy_inference_layer_table['layer_voltage_mean'] = layer_voltages_mean.mean(axis=1)
        energy_inference_layer_table['layer_voltage_std'] = layer_voltages_std.mean(axis=1)
        energy_inference_layer_table['layer_current_mean'] = layer_currents_mean.mean(axis=1)
        energy_inference_layer_table['layer_current_std'] = layer_currents_std.mean(axis=1)
        energy_inference_layer_table['layer_power_mean'] = layer_powers_mean.mean(axis=1)
        energy_inference_layer_table['layer_power_std'] = layer_powers_std.mean(axis=1)
        energy_inference_layer_table['layer_energy_mean'] = layer_energies.mean(axis=1)
        energy_inference_layer_table['layer_energy_std'] = layer_energies.std(axis=1)


        energy_inference_layer_table.to_pickle(f"results_energy/{model_information['MCU']}/"
                                        f"{model_information['MCU']}_"
                                        f"{model_information['model_name']}_"
                                        f"{model_information['model_type']}_"
                                        f"{model_information['cmsis-nn']}_"
                                         f"energy-layer_results_{date.today()}.pkl")
        #energy_inference_layer_table.to_excel(f"results_energy/{model_information['MCU']}_{model_information['model']}_energy-layer_results_{date.today()}.xlsx")


    # print(inference_latencies, "s")
    # print(inference_voltages_mean, "V")
    # print(inference_currents_mean,"mA")
    # print(inference_powers_mean, "W")
    # print(inference_energies, "J")


    # save infererence data in a dict
    inference_dict['latency_mean'] = inference_latencies.mean()
    inference_dict['latency_std'] = inference_latencies.std()
    inference_dict['voltage_mean'] = inference_voltages_mean.mean()
    inference_dict['voltage_std'] = inference_voltages_std.mean()
    inference_dict['current_mean'] = inference_currents_mean.mean()
    inference_dict['current_std'] = inference_currents_std.mean()
    inference_dict['power_mean'] = inference_powers_mean.mean()
    inference_dict['power_std'] = inference_powers_std.mean()
    inference_dict['energy_mean'] = inference_energies.mean()
    inference_dict['energy_std'] = inference_energies.std()

    # add dict to table
    energy_inference_table = energy_inference_table.append(inference_dict, ignore_index=True)

    energy_inference_table.to_pickle(f"results_energy/{model_information['MCU']}/"
                                    f"{model_information['MCU']}_"
                                    f"{model_information['model_name']}_"
                                    f"{model_information['model_type']}_"
                                    f"{model_information['cmsis-nn']}_"
                                     f"energy_results_{date.today()}.pkl")

    #energy_inference_table.to_excel(f"results_energy/{model_information['MCU']}_{model_information['model']}_energy_results_{date.today()}.xlsx")


    print("\nDoing some sanity checks ...")
    print(f"Sum of the average layer latencies:\t{energy_inference_layer_table['layer_latency_mean'].sum()}")
    print(f"Average inference latency:\t\t{energy_inference_table['latency_mean'][0]}")

    print(f"Sum of the average layer energies:\t{energy_inference_layer_table['layer_energy_mean'].sum()}")
    print(f"Average inference energy:\t\t{energy_inference_table['energy_mean'][0]}")
    
    return energy_inference_table, energy_inference_layer_table

## Some manual checks for the just analysed data

In [90]:
energy_inference_table

,time,filename,MCU,model,mbed-dir,cmsis-nn,compiler_optimization,FPU_status,model_type,weights,...,latency_std,voltage_mean,voltage_std,current_mean,current_std,power_mean,power_std,energy_mean,energy_std,model_name
0,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000142,3.305771,0.001435,0.068901,0.001715,0.227771,0.00565,0.223953,0.000142,01d_ResNet20_CIFAR-10


In [91]:
energy_inference_layer_table

,time,filename,MCU,model,mbed-dir,cmsis-nn,compiler_optimization,FPU_status,model_type,weights,...,layer_latency_std,layer_voltage_mean,layer_voltage_std,layer_current_mean,layer_current_std,layer_power_mean,layer_power_std,layer_energy_mean,layer_energy_std,model_name
0,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000009,3.305598,0.001425,0.070125,0.002756,0.231801,0.009081,0.000185,0.000002,01d_ResNet20_CIFAR-10
1,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000017,3.305372,0.001407,0.071383,0.000576,0.235944,0.001898,0.005163,0.000011,01d_ResNet20_CIFAR-10
2,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000018,3.305874,0.001406,0.068408,0.000633,0.226145,0.002087,0.013067,0.000018,01d_ResNet20_CIFAR-10
3,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000013,3.305746,0.001412,0.069081,0.000700,0.228362,0.002305,0.013213,0.000017,01d_ResNet20_CIFAR-10
4,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000059,3.305369,0.001403,0.071545,0.000330,0.236480,0.001090,0.003773,0.000020,01d_ResNet20_CIFAR-10
5,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000017,3.305779,0.001412,0.068959,0.000639,0.227962,0.002106,0.013193,0.000015,01d_ResNet20_CIFAR-10
6,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000014,3.305840,0.001408,0.068566,0.000711,0.226667,0.002344,0.013110,0.000019,01d_ResNet20_CIFAR-10
7,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000052,3.305380,0.001410,0.071385,0.000334,0.235954,0.001104,0.003735,0.000015,01d_ResNet20_CIFAR-10
8,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000009,3.305766,0.001409,0.069061,0.000852,0.228297,0.002807,0.013221,0.000017,01d_ResNet20_CIFAR-10
9,2020-08-05 16:24:54.790999,energy_measurements/20200805_142649_inference-...,DISCO_F469NI,01d_ResNet20_CIFAR-10_none_tflite-builtins-INT...,./TFLu_benchmark-model_mbed_cmsis-nn,cmsis-nn,-Ofast,1,W-int8_A-int8,int8,...,0.000007,3.305815,0.001410,0.068714,0.000659,0.227154,0.002170,0.013142,0.000008,01d_ResNet20_CIFAR-10


In [13]:
energy_inference_layer_table['layer_latency_mean'].sum()

0.036388906249999874

In [279]:
energy_inference_layer_table['layer_latency_mean']

0     0.109883
1     0.382381
2     0.382245
3     0.001834
4     0.382380
5     0.382380
6     0.001836
7     0.382378
8     0.382249
9     0.001831
10    0.025055
11    0.193568
12    0.349938
13    0.000915
14    0.349937
15    0.349935
16    0.000917
17    0.349936
18    0.349938
19    0.000914
20    0.179721
21    0.320023
22    0.022125
23    0.000455
24    0.320027
25    0.319924
26    0.000456
27    0.320025
28    0.319925
29    0.000454
30    0.000471
31         NaN
32         NaN
Name: layer_latency_mean, dtype: float64

In [80]:
print(1/64000)

1.5625e-05


In [44]:
layer_latencies.mean(axis=1).sum()

0.12044640625000058

In [15]:
layer_energies.mean(axis=1).sum()

0.00157919238015111

In [137]:
df = energy_inference_layer_table
file = 'L4_LeNet-MNIST_int8_non-cmsis-nn_layers'
with open(f'{file}.md', 'w') as f:
    f. write(df.to_markdown())

In [137]:
df = energy_inference_layer_table
file = 'L4_LeNet-MNIST_int8_non-cmsis-nn_layers'
with open(f'{file}.md', 'w') as f:
    f. write(df.to_markdown())